# Answering Business Questions using SQL

In this project, will use a Chinook database. The Chinook database is provided as a SQLite database file called `chinook.db`. A copy of the database schema is below - I will need to come back to consult the schema while writing the queries. ![Image](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Let's start by getting familiar with our data.

To run SQL queries in Jupyter Notebook, we have to add `%%sql` on its own line to the start of our query

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT
    *
FROM sqlite_master
LIMIT 10

 * sqlite:///chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


The Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:


* **Regal = Hip-Hop**
* **Red Tone = Punk**
* **Meteor and the Girls = Pop**
* **Slim Jim Bites = Blues**

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

## A query that returns each genre, with the number of tracks sold in the USA

I will write a query to find out which genres sell the most tracks in the USA, write up a summary of my findings, and make a recommendation for the three artists whose albums that should be purchased for the store.

This query will include the number of tracks sold in the USA in:
* in absolute numbers
* in percentages.

In [4]:
%%sql
WITH in_usa AS
    (
    SELECT *
    FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
    )
    
SELECT a.name artist_name, 
       g.name genre,
       t.composer composer,
       COUNT(iu.track_id) track_sold_usa,
       ROUND(CAST(COUNT(iu.track_id) AS Float)/
        (SELECT COUNT(track_id) FROM in_usa), 4) percentage_usa_sold 
FROM in_usa iu
INNER JOIN track t ON t.track_id = iu.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist a ON a.artist_id = al.artist_id
GROUP BY genre
ORDER BY track_sold_usa DESC
LIMIT 10;
    

 * sqlite:///chinook.db
Done.


artist_name,genre,composer,track_sold_usa,percentage_usa_sold
The Who,Rock,Pete Townshend,561,0.5338
Green Day,Alternative & Punk,Billie Joe Armstrong -Words Green Day -Music,130,0.1237
Godsmack,Metal,Sully Erna,124,0.118
Amy Winehouse,R&B/Soul,None,53,0.0504
Buddy Guy,Blues,Robert Geddins/Ron Badger/Sheldon Feinberg,36,0.0343
Chris Cornell,Alternative,Chris Cornell,35,0.0333
Eric Clapton,Latin,Gilberto Gil,22,0.0209
U2,Pop,None,22,0.0209
House Of Pain,Hip Hop/Rap,L. Muggerud,20,0.019
Miles Davis,Jazz,"H. Spina, J. Elliott",14,0.0133


Based on the findings, I would recommended purchasing the new albums from these artists:
* The Who (Rock)
* Green Day (Alternate & Punk)
* Godsmack (Metal)

I would strongly advise on looking out for artists in the rock genre as they make about 53% of sales in addition to composers in that genre as well.

## A query that finds the total dollar amount of sales assigned to each sales support agent within the company.

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. Next, is to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

I will use any extra attributes for that employee that I find relevant to the analysis.

In [5]:
%%sql
SELECT 
      e.first_name || " " || e.last_name employee_name,
      c.first_name || " " || c.last_name customer_name,
      e.title employee_title,
      e.hire_date,
      ROUND(SUM(i.total), 2) total_sales
FROM invoice i
INNER JOIN customer c ON c.customer_id = i.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY employee_name
ORDER BY total_sales DESC

 * sqlite:///chinook.db
Done.


employee_name,customer_name,employee_title,hire_date,total_sales
Jane Peacock,Phil Hughes,Sales Support Agent,2017-04-01 00:00:00,1731.51
Margaret Park,Dan Miller,Sales Support Agent,2017-05-03 00:00:00,1584.0
Steve Johnson,Mark Philips,Sales Support Agent,2017-10-17 00:00:00,1393.92


From the analysis, Jane (top employee) has the highest total sales and Steve with the lowest. There is a positive correlation between the date the sales support agents were hired and the total sales as the one hired for longer has the most total sales

## A query that collates data on purchases from different countries

The next task is to analyze the sales data for customers from each different country. I have been given guidance to use the country value from the `customers` table, and ignore the country from the billing address in the `invoice` table.

A query that collates data on purchases from different countries:

* Where a country has only one customer, this will be collected into an "Other" group.

* The results will be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

* For each country, will include:
    * total number of customers
    * total value of sales
    * average value of sales per customer
    * average order value



In [66]:
%%sql
WITH other AS 
    (
    SELECT *
           
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    )
     
SELECT
       COUNT(DISTINCT o.customer_id) "total number of customers",
       ROUND(SUM(o.unit_price),2) "total value of sales",
       ROUND(CAST(SUM(o.unit_price) AS Float)/COUNT(DISTINCT o.customer_id), 2) "average value of sales per customer",
       ROUND((SUM(o.unit_price))/COUNT(DISTINCT o.invoice_id),2) "average order value",
       o.country country,
       CASE
            WHEN 
            (SELECT COUNT (c.first_name || " " || c.last_name)
            FROM customer c
            WHERE c.country = o.country
            ) = 1 THEN "other"
            ELSE 0
       END AS sort
FROM other o       

GROUP BY country
ORDER BY "total value of sales" DESC

 * sqlite:///chinook.db
Done.


total number of customers,total value of sales,average value of sales per customer,average order value,country,sort
13,1040.49,80.04,7.94,USA,0
8,535.59,66.95,7.05,Canada,0
5,427.68,85.54,7.01,Brazil,0
5,389.07,77.81,7.78,France,0
4,334.62,83.66,8.16,Germany,0
2,273.24,136.62,9.11,Czech Republic,0
3,245.52,81.84,8.77,United Kingdom,0
2,185.13,92.57,6.38,Portugal,0
2,183.15,91.58,8.72,India,0
1,114.84,114.84,8.83,Ireland,other


Based on the data, there could be opportunities in:

* Czech Republic
* United Kingdom
* Germany
* India

But please bear in mind that this data is based off a small sample size and may not reflect analysis done on a much larger scale. At best, it will important to conduct small campaigns in these countries to confirm the validity of this data before pursuing.

## A query that categorizes each invoice as either an album purchase or not

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

I have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

Since our analysis is concerned with maximizing revenue, I will focus on customers who may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In order to answer the question, I to have to identify whether each invoice has all the tracks from an album. I can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. I can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

Also I am going to calculate the following summary statistics:

* Number of invoices
* Percentage of invoices

In [68]:
%%sql
WITH invoice_first_track AS
    (
     SELECT 
        il.invoice_id invoice_id,
        il.track_id first_track_id
    FROM invoice_line il
    GROUP BY 1
    )

SELECT purchased_album,
    COUNT(invoice_id) "Number of invoices",
    ROUND(CAST(COUNT(invoice_id) AS Float)/
                                    (
                                    SELECT COUNT(i.invoice_id)
                                    FROM invoice i 
                                    ),3) 'Percentage of invoices'
FROM
    (
    SELECT *,
        CASE
            WHEN 
                (
                SELECT t.track_id FROM track t
                WHERE t.album_id = 
                    (
                    SELECT t2.album_id FROM track t2
                    WHERE t2.track_id = ift.first_track_id
                    ) 
                EXCEPT
                SELECT il2.track_id FROM invoice_line il2
                WHERE il2.invoice_id = ift.invoice_id
                )
                IS NULL
            
            AND

                (
                SELECT il2.track_id FROM invoice_line il2
                WHERE il2.invoice_id = ift.invoice_id
                EXCEPT
                SELECT t.track_id FROM track t
                WHERE t.album_id = 
                    (
                    SELECT t2.album_id FROM track t2
                    WHERE t2.track_id = ift.first_track_id
                    )
                ) 
                IS NULL 
        
            THEN "Yes"
            ElSE "No"
        END AS purchased_album
    FROM invoice_first_track ift
    )
GROUP BY purchased_album;

 * sqlite:///chinook.db
Done.


purchased_album,Number of invoices,Percentage of invoices
No,500,0.814
Yes,114,0.186


Purchased albums accounts for 18.6% of purchases. I would recommend the company purchase every track from an album so as to not lose the fifth of the revenue - 18.6%.